<a href="https://colab.research.google.com/github/wael-slimi/Speciality_Recommedation/blob/main/Disease%2C_Speciality_Recommedation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'doctor-specialist-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3417941%2F5969573%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240425%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240425T020157Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D10edc02ee98f39bb687a0e1c762b67d5ebd87ce4f6bf704f1d33de5ecfc7690e6924588b881146388e164fce78529ea2a769f61da3f56cc2fca13be64e609618065f48bec268eab39cf67149ca7c4df4da0e4167b5ffdf30fb4d0a6a4184425ebaa1ae0dfca892445cf7ddc76df4a88fea34fb6a212fff9cab643dacf9ca25ffe276a5d54d4bcd88ed3e7ce7a6edf7b595a9b436a918c7ddb5a98c8ce904e203f5e111aebdfcacea61f0185a55b76224706719ab84b04ab2fbfb8ffea4aa1d17fa5612740ca811c1aecfad98e5194bc987759798b72ed00378f1f1b544fd49396633a496043573bac6a1d135fa2931bd680cf8895ea801fe5c1a472807fde133'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 942184 bytes downloaded
Downloaded and uncompressed: doctor-specialist-recommendation-system
Data source import complete.


### Predicting Disease based on the Symptoms and identifying specialist based on predicted disease

In [ ]:
! cd

**The dataset contains information about symptoms, diseases, recommended doctors for each disease, and descriptions of the diseases. Using a classification model, diseases have been predicted based on the given symptoms. The predictions provide insights into the predicted disease, the likelihood or chance of that disease based on the classification model, the recommended doctor to visit for that disease, and a description of the predicted disease.**

**The summary output includes:**

**Predicted Disease:** *-The disease that has been predicted based on the symptoms provided.-*

**Chance of the Disease:** *-The likelihood or probability of having that disease, as determined by the classification model.-*

**Recommended Doctor:** *-The doctor or specialist recommended to visit for further evaluation or treatment of the predicted disease.-*

**Description of the Disease:** *-A brief overview or description of the predicted disease, providing additional information about its symptoms, causes, and potential treatments.-*

**This summary aims to provide a concise and informative overview of the predicted disease, its associated likelihood, the recommended doctor, and a brief description to aid in understanding and decision-making regarding the predicted disease.**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [5]:
pd.set_option('display.max_colwidth', 50)

In [6]:
dis_sym_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Original_Dataset.csv")

In [7]:
dis_sym_data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dis_sym_data.shape

(4920, 18)

**Finding unique values across all the symptoms column**

In [9]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [10]:
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

**Convert Symptoms to Binary Columns**

In [11]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)

In [12]:
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]

In [13]:
dis_sym_data_v1.shape

(4920, 132)

In [14]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()

In [15]:
dis_sym_data_v1.columns

Index(['Disease', 'stomach_pain', 'headache', 'malaise', 'stiff_neck',
       'painful_walking', 'brittle_nails', 'skin_rash', 'muscle_pain',
       'extra_marital_contacts',
       ...
       'small_dents_in_nails', 'passage_of_gases', 'high_fever',
       'rusty_sputum', 'mild_fever', 'pain_during_bowel_movements', 'anxiety',
       'swelling_joints', 'visual_disturbances', 'fatigue'],
      dtype='object', length=132)

**Encoding Disease**

In [16]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])

In [17]:
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

**Model Generation**

In [18]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [19]:
algorithms = {'Logistic Regression':
              {"model": LogisticRegression()},

              'Decision Tree':
              {"model": tree.DecisionTreeClassifier()},

              'Random Forest':
              {"model": RandomForestClassifier()},

              'SVM':
              {"model": svm.SVC(probability=True)},

              'NaiveBayes' :
              {"model": GaussianNB()},

              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [20]:
for model_name, values in algorithms.items():
    class_algo(values["model"],X,y)

Logistic Regression Accuracy : 100.000%
Decision Tree Accuracy : 100.000%
Random Forest Accuracy : 100.000%
SVM Accuracy : 100.000%
NaiveBayes Accuracy : 100.000%
K-Nearest Neighbors Accuracy : 100.000%


**Map Description and Specialist for the Disease Predicted**

In [21]:
doc_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [22]:
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Tuberculosis


 -*Doctor name is wrong for Disease - Tuberculosis. Replace to Pulmonologist*-

In [23]:
doc_data['Specialist'] = np.where((doc_data['Disease'] == 'Tuberculosis'),'Pulmonologist', doc_data['Specialist'])

In [ ]:
doc_data.tail(5)

In [25]:
des_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Disease_Description.csv")

In [26]:
des_data.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


**Test with Unknown Data**

In [27]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

In [31]:
# Looping through each column name in the DataFrame dis_sym_data_v1
# Each column represents a symptom
for sym in dis_sym_data_v1:
    # Printing each column name (which represents a symptom)
    print(sym)


Disease
stomach_pain
headache
malaise
stiff_neck
painful_walking
brittle_nails
skin_rash
muscle_pain
extra_marital_contacts
weight_gain
altered_sensorium
red_sore_around_nose
red_spots_over_body
acidity
cold_hands_and_feets
abdominal_pain
distention_of_abdomen
neck_pain
loss_of_smell
swelling_of_stomach
pain_in_anal_region
dark_urine
receiving_unsterile_injections
sunken_eyes
drying_and_tingling_lips
sweating
sinus_pressure
puffy_face_and_eyes
bladder_discomfort
blurred_and_distorted_vision
blood_in_sputum
irritability
burning_micturition
chills
watering_from_eyes
cough
bruising
hip_joint_pain
acute_liver_failure
obesity
continuous_sneezing
foul_smell_of urine
dehydration
swollen_blood_vessels
yellow_urine
enlarged_thyroid
weakness_of_one_body_side
muscle_weakness
lethargy
fluid_overload
palpitations
yellowish_skin
silver_like_dusting
indigestion
prominent_veins_on_calf
knee_pain
excessive_hunger
lack_of_concentration
increased_appetite
skin_peeling
breathlessness
shivering
irregular_s

In [32]:
test_input()

Enter the number of symptoms you have: 3
Enter Symptoms #1: cough
Enter Symptoms #2: swollen_blood_vessels
Enter Symptoms #3: weakness_of_one_body_side
Symptoms you have: ['cough', 'swollen_blood_vessels', 'weakness_of_one_body_side']
Predicting Disease based on 6 ML algorithms...


,Disease,Chances,Specialist,Description
0,Paralysis (brain hemorrhage),83.333333,Neurologist,Intracerebral hemorrhage (ICH) is when blood s...
1,Cervical spondylosis,16.666667,Neurologist,Cervical spondylosis is a general term for age...


In [ ]:
!ls


.























.

In [36]:
import random

# Generate random latitude and longitude within the range of [-90, 90] and [-180, 180] respectively
user_latitude = random.uniform(-90, 90)
user_longitude = random.uniform(-180, 180)

print(f"Random User Location - Latitude: {user_latitude}, Longitude: {user_longitude}")


Random User Location - Latitude: 44.28190206834597, Longitude: 14.47378983434055


In [42]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in km
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Compute the differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula to calculate the distance
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
data = pd.read_csv("/content/sample_data/doctor_data.csv")

# Assuming the dataset contains doctor locations (latitude and longitude)

# User location input (latitude and longitude)
user_latitude = float(input("Enter your latitude: "))
user_longitude = float(input("Enter your longitude: "))

# Calculate distances between user location and doctor locations
data['Distance'] = data.apply(lambda row: calculate_distance(user_latitude, user_longitude, row['Doctor_Latitude'], row['Doctor_Longitude']), axis=1)

# Sort doctors by distance
data.sort_values(by='Distance', inplace=True)

# Display recommended doctors with distances in kilometers
recommended_doctors = data[['Doctor', 'Specialty', 'Distance']].head(5)
recommended_doctors['Distance'] = recommended_doctors['Distance'].apply(lambda x: f"{x:.2f} km")
print("Recommended Doctors:")
print(recommended_doctors)
